<a href="https://colab.research.google.com/github/s11000/pandas-data-frame/blob/main/G100A007_0610ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
#include <math.h>

#define B1  4
#define B2  7
#define LED 13

#define PERCEPTRON  3

int X[1][2]  = { {1,0} };

/*these matrices was calculated by python */
float W1[2][PERCEPTRON] =   {
                    {0.38492545, 3.94078829, 4.24865286}, 
                    {3.83936678, 0.20450267, 4.27767947}
                   };
float W2[PERCEPTRON][1] =   { {-2.6730679}, 
                     {-2.3817902}, 
                    {4.99647104} };
float Wo1[1][PERCEPTRON];
float sum = 0;
float Y = 0;

/*sigmoid function*/
float sigmoid (float x)
{
    return 1/(1 + exp(-x));
}

void setup()
{
  Serial.begin(115200);

  pinMode(B1, INPUT); 
  pinMode(B2, INPUT); 
  pinMode(LED, OUTPUT);

  digitalWrite(LED, LOW);
}

void loop()
{
  X[0][0] = digitalRead(B1);
  X[0][1] = digitalRead(B2);
  Serial.print("Button 1:"); Serial.print(X[0][0]);
  Serial.print("\tButton 2:"); Serial.println(X[0][1]);

  /* calculate forward part based on weights */
  //hidden layer
  for(int i=0; i<1; i++)
  {
      for(int j=0;j <PERCEPTRON; j++)
      {
          for(int k=0; k<2; k++)
          {
              sum += X[i][k]*W1[k][j];
          }
          Wo1[i][j] = sigmoid(sum);
          sum = 0;  
      }
  }
  //output layer
  for(int i=0; i<1; i++)
  {
      for(int j=0;j <1; j++)
      {
          for(int k=0; k<PERCEPTRON; k++)
          {
              Y += Wo1[i][k]*W2[k][j];
          } 
      }
  }

  Serial.println(Y, 2);

  Y = round(Y);
  if (int(Y) == 1)
    Serial.println("---- Should be... ON ----");
  else
    Serial.println("---- Should be... OFF ----");
  digitalWrite(LED, int(Y));
  Y = 0;

  delay(2000);
}

SyntaxError: ignored

In [5]:
import numpy as np


# Times to run
epoch = 10000


# There are 2 inputs
inputLayerSize = 2


# NN nodes
hiddenLayerSize = 3


# Only one output
outputLayerSize = 1


L=0.1


# There are 2 inputs for XOR
X = np.array( [ [0,0], [0,1], [1,0], [1,1] ] )


# The truth table of XOR
# ANN just can learn from truly examples!!!
#     (adjust the weight and bias to make output is getting close to target by input)
Y = np.array( [ [0], [1], [1], [0] ] )


def sigmod(x):
    return 1 / (1 + np.exp(-x))


def sigmoid_deriv(x):
	return x * (1 - x)
	
Wh = np.random.uniform(size=(inputLayerSize, hiddenLayerSize))
Wz = np.random.uniform(size=(hiddenLayerSize, outputLayerSize))


for i in range(epoch):
  H = sigmod(np.dot(X, Wh))
  Z = np.dot(H, Wz)
  E = Y - Z
  dZ = E * L
  Wz += H.T.dot(dZ)
  dH = dZ.dot(Wz.T) * sigmoid_deriv(H)
  Wh +=  X.T.dot(dH)


print("**************** error ****************") 
print(E)
print("***************** output **************") 
print(Z)   
print("*************** weights ***************") 
print("input to hidden layer weights: ")     
print(Wh)
print("hidden to output layer weights: ")
print(Wz)

**************** error ****************
[[-3.77814599e-07]
 [ 4.56158989e-07]
 [ 5.04131912e-07]
 [-6.94601447e-07]]
***************** output **************
[[3.77814599e-07]
 [9.99999544e-01]
 [9.99999496e-01]
 [6.94601447e-07]]
*************** weights ***************
input to hidden layer weights: 
[[ 4.37953108  0.0306285   0.92571624]
 [ 4.32386247 -1.62363698  0.57830025]]
hidden to output layer weights: 
[[ 4.69344926]
 [ 1.31331238]
 [-6.00676088]]


In [6]:
import numpy as np
from keras.datasets import mnist
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import SimpleRNN, Activation, Dense
from keras.optimizers import Adam

# 固定亂數種子，使每次執行產生的亂數都一樣
np.random.seed(1337)


# 載入 MNIST 資料庫的訓練資料，並自動分為『訓練組』及『測試組』
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# 將 training 的 input 資料轉為3維，並 normalize 把顏色控制在 0 ~ 1 之間
X_train = X_train.reshape(-1, 28, 28) / 255.      
X_test = X_test.reshape(-1, 28, 28) / 255.
y_train = np_utils.to_categorical(y_train, num_classes=10)
y_test = np_utils.to_categorical(y_test, num_classes=10)


# 建立簡單的線性執行的模型
model = Sequential()
# 加 RNN 隱藏層(hidden layer)
model.add(SimpleRNN(
    # 如果後端使用tensorflow，batch_input_shape 的 batch_size 需設為 None.
    # 否則執行 model.evaluate() 會有錯誤產生.
    batch_input_shape=(None, 28, 28), 
    units= 50,
    unroll=True,
)) 
# 加 output 層
model.add(Dense(units=10, kernel_initializer='normal', activation='softmax'))

# 編譯: 選擇損失函數、優化方法及成效衡量方式
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) 

# 一批訓練多少張圖片
BATCH_SIZE = 50     
BATCH_INDEX = 0     
# 訓練模型 4001 次
for step in range(1, 4001):
    # data shape = (batch_num, steps, inputs/outputs)
    X_batch = X_train[BATCH_INDEX: BATCH_INDEX+BATCH_SIZE, :, :]
    Y_batch = y_train[BATCH_INDEX: BATCH_INDEX+BATCH_SIZE, :]
    # 逐批訓練
    loss = model.train_on_batch(X_batch, Y_batch)
    BATCH_INDEX += BATCH_SIZE
    BATCH_INDEX = 0 if BATCH_INDEX >= X_train.shape[0] else BATCH_INDEX

    # 每 500 批，顯示測試的準確率
    if step % 500 == 0:
        # 模型評估
        loss, accuracy = model.evaluate(X_test, y_test, batch_size=y_test.shape[0], 
            verbose=False)
        print("test loss: {}  test accuracy: {}".format(loss,accuracy))
        

# 預測(prediction)
X = X_test[0:10,:]
predictions = model.predict_classes(X)
# get prediction result
print(predictions)

# 模型結構存檔
from keras.models import model_from_json
json_string = model.to_json()
with open("SimpleRNN.config", "w") as text_file:
    text_file.write(json_string)
    
# 模型訓練結果存檔
model.save_weights("SimpleRNN.weight")

ImportError: ignored